In [1]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

2025-05-17 23:31:00.796249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747524660.990557      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747524661.048697      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

In [5]:
wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='qwen-3-1.7b wealth management chatbot finetune', 
    job_type="training", 
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: youngrha430 (youngrha430-sekure-payment-experts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.11
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250517_233119-uj4sd0dw
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run cosmic-brook-2
wandb: ⭐️ View project at https://wandb.ai/youngrha430-sekure-payment-experts/qwen-3-1.7b%20wealth%20management%20chatbot%20finetune?apiKey=3ff0b518def644e46cfac36e679067b7eb574d13
wandb: 🚀 View run at https://wandb.ai/youngrha430-sekure-payment-experts/qwen-3-1.7b%20wealth%20management%20chatbot%20fi

In [6]:
base_model = "/kaggle/input/qwen-3/transformers/1.7b/1"
new_model = "qwen-3-1.7b-wealth-finetune"
dataset_name = "bitext/Bitext-wealth-management-llm-chatbot-training-dataset"

In [7]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={'': torch.cuda.current_device()},
    attn_implementation=attn_implementation,
)

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

print(f"Model is loaded on device: {next(model.parameters()).device}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model is loaded on device: cuda:0


In [9]:
from datasets import get_dataset_config_names

print(get_dataset_config_names(dataset_name))

README.md:   0%|          | 0.00/9.81k [00:00<?, ?B/s]

['default']


In [10]:
config_name = "default"

In [11]:
dataset_dict = load_dataset(dataset_name, name=config_name)
print(dataset_dict)

(…)agement-llm-chatbot-training-dataset.csv:   0%|          | 0.00/25.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/23806 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['system_prompt', 'instruction', 'intent', 'category', 'tags', 'response'],
        num_rows: 23806
    })
})


In [12]:
dataset = dataset_dict["train"]
dataset = dataset.shuffle(seed=65).select(range(1000))
instruction = """You are a wealth management assistant. 
     Be professional be matter of fact about your points.
    """
def format_chat_template(row):
    row_json = [
        {
            "role": "user",
            "content": instruction + "\n\n" + row["instruction"],  # system + user combined
        },
        {
            "role": "assistant",
            "content": row["response"],
        },
    ]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row



dataset = dataset.map(
    format_chat_template,
    num_proc= 4,
)


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
train_val = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_val['train']
val_dataset = train_val['test']

In [14]:
dataset

Dataset({
    features: ['system_prompt', 'instruction', 'intent', 'category', 'tags', 'response', 'text'],
    num_rows: 1000
})

In [15]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

In [16]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)
tokenizer.chat_template = None
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

In [17]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=20,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [18]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    
    args=training_arguments,
    
)

Converting train dataset to ChatML:   0%|          | 0/900 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [19]:
print(torch.cuda.device_count())
print(torch.cuda.current_device())

1
0


In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
20,1.208600,1.085535
40,1.198000,0.858239
60,1.016300,0.822877
80,0.739000,0.768913
100,0.983600,0.772815
120,0.816500,0.734541
140,0.646400,0.720820
160,0.723300,0.717902
180,0.657200,0.687739
200,0.933000,0.685288


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=450, training_loss=0.7499261670642429, metrics={'train_runtime': 688.2985, 'train_samples_per_second': 1.308, 'train_steps_per_second': 0.654, 'total_flos': 1810029771730944.0, 'train_loss': 0.7499261670642429})

In [21]:
wandb.finish()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                 eval/loss █▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
wandb:  eval/mean_token_accuracy ▁▄▅▆▅▆▆▆▇▇▇▇▇▇▇▇██████
wandb:           eval/num_tokens ▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
wandb:              eval/runtime ▁▃█▅▅▅▅▄▅▄▅▅▅▅▅▆▅▄▅▅▅▅
wandb:   eval/samples_per_second █▆▁▄▄▄▄▅▃▄▄▄▄▄▄▃▄▅▄▄▄▄
wandb:     eval/steps_per_second █▆▁▄▄▄▄▅▃▄▄▄▄▄▄▃▄▅▄▄▄▄
wandb:               train/epoch ▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇███
wandb:         train/global_step ▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇█
wandb:           train/grad_norm ▄▃▄▃▅▇▂▄█▂▂▄▂▁▁▂▁▂▂▂▂▃▂▁▂▂▅▂▃▂▅▁▂▂▁▃▄▂▂▃
wandb:       train/learning_rate ▃▅██████▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁
wandb:                train/loss █▆█▄▅▂▃▆▄▂▄▃▄▂▂▄▄▂▄▂▁▃▂▃▂▃▄▇▁▃▃▁▂▂▃▃▁▄▃▃
wandb: train/mean_token_accuracy ▁▆▃▄▂▃▆▇▃▆▄▅▃▆▄▆▇█▃▅█▆▆▅█▇▆▆▃▆▆▆██▆█▅▆█▆
wandb:          train/num_tokens ▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
wandb: 
wandb: Run summary:
wan

In [22]:
messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "I want to start invest in stock. how should I proceed?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150)

generated_tokens = outputs[0][inputs['input_ids'].shape[-1]:]

response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(response.strip())

<think>

</think>

I'll make it happen! Investing in stocks can be a great way to grow your wealth. Here's a step-by-step guide on how to get started:

1. Determine your investment goals: Think about how much money you want to invest and for how long. Are you looking for short-term gains or long-term growth?

2. Choose a brokerage firm: Research different brokerage firms and select one that fits your needs and preferences. Consider factors like fees, available services, and user-friendly platforms.

3. Open an account: Visit the website or go to the store of your chosen brokerage firm. Fill out the registration form with your personal information and choose a secure password.

4. Fund your account: Deposit the amount you want to


In [23]:
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/young43/qwen-3-1.7b-wealth-finetune/commit/533cb4169f3fa02287522d3e52875de3fceb16da', commit_message='Upload model', commit_description='', oid='533cb4169f3fa02287522d3e52875de3fceb16da', pr_url=None, repo_url=RepoUrl('https://huggingface.co/young43/qwen-3-1.7b-wealth-finetune', endpoint='https://huggingface.co', repo_type='model', repo_id='young43/qwen-3-1.7b-wealth-finetune'), pr_revision=None, pr_num=None)